In [ ]:
# 导入读取数据所需要的库
import numpy as np
import pandas as pd
from pandas import read_excel

df = read_excel(r'data\SEV_wordlist.xlsx')

In [1]:
# 获取Excel文件中所有表的名称
excel_file = pd.ExcelFile(r'data\SEV_wordlist.xlsx')
print(excel_file.sheet_names)  # 显示所有表的名称


['App_words', 'SES_words', 'Soc_words', 'Com_words', 'Mor_words']


In [3]:
import gensim
from gensim import models

In [4]:
# 预训练的词向量语料库文件路径
tecent_embedding_path = (r'D:\embedding_zh\tencent-ailab-embedding-zh-d200-v0.2.0-s\tencent-ailab-embedding-zh-d200-v0.2.0-s.txt')

# 加载词向量文件
tencent_embedding = models.KeyedVectors.load_word2vec_format(tecent_embedding_path, binary=False)

In [6]:
# SES扩展词表

df_SES_words = pd.read_excel(r'data\SEV_wordlist.xlsx', sheet_name=1)  # 表的索引从0开始，1表示第二张表

# 查看数据的形状
df_SES_words.shape

# 预览数据
df_SES_words.head()

,Word,Dimension,SES_Dim_mean,SES_Dim_sd,App_Dim_mean,App_Dim_sd,Soc_Dim_mean,Soc_Dim_sd,Com_Dim_mean,Com_Dim_sd,...,Val_sd,Valence,N_raters,N_App,N_SES,N_Soc,N_Com,N_Mor,N_Non,N_Amb
0,首富,Socioeconomic status,8.268,1.977,2.009569,1.734798,2.688995,2.323360,4.952153,3.195190,...,1.653,Positive,209,3,148,2,9,1,6,40
1,上流,Socioeconomic status,8.161,1.741,3.654028,2.652711,4.331754,2.907115,4.037915,2.933953,...,1.495,Positive,211,0,144,4,6,0,8,49
2,富裕,Socioeconomic status,8.090,1.968,2.535545,2.122985,2.507109,2.127752,4.364929,3.014963,...,1.531,Positive,211,2,155,1,11,0,9,33
3,土豪,Socioeconomic status,8.066,1.683,3.260664,2.448190,2.800948,2.327516,3.829384,2.811420,...,1.441,Positive,211,2,163,2,7,1,4,32
4,富翁,Socioeconomic status,7.981,2.138,2.590698,2.320149,2.730233,2.453639,3.739535,2.966648,...,1.780,Positive,215,1,154,2,3,0,18,37


In [8]:
# 求积极词汇的词表
df_SES_positive = df_SES_words.loc[df_SES_words['Valence']=='Positive', 'Word']
df_SES_positive.head()

0    首富
1    上流
2    富裕
3    土豪
4    富翁
Name: Word, dtype: object

In [9]:
# 格式转化为列表
SES_positive_word_list = df_SES_positive.tolist()
SES_positive_word_list

['首富',
 '上流',
 '富裕',
 '土豪',
 '富翁',
 '巨富',
 '显赫',
 '富豪',
 '尊贵',
 '功成名就',
 '腰缠万贯',
 '富贵',
 '富有',
 '有名',
 '财富',
 '富足',
 '殷富',
 '高层',
 '高人一等',
 '风云人物',
 '有头有脸',
 '身价',
 '华贵',
 '著名',
 '权威',
 '豪气',
 '阔绰',
 '阔气',
 '高贵',
 '家喻户晓',
 '农民',
 '煊赫',
 '贵气',
 '体面',
 '殷实',
 '教授',
 '举足轻重',
 '优厚',
 '气派',
 '勤工俭学',
 '默默无闻']

In [11]:
# 初始化有效种子词列表和相似词列表
SES_positive_word_list_effective = []
SES_similar_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in SES_positive_word_list:
    if word in tencent_embedding:
        SES_positive_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        SES_similar_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_word, similarity in SES_similar_words:
            SES_similar_words_list.append(similar_word)
    else:
        print(f"Skipping {word}...")

In [12]:
# 将生成的积极相似词先转换为 DataFrame，查看是否生成成功
df_SES_similar_words = pd.DataFrame(SES_similar_words_list, columns=['Word'])
print(df_SES_similar_words)

     Word
0    全国首富
1    第一首富
2    第一富豪
3      巨富
4    中国首富
..    ...
405  寂寂无闻
406  藉藉无名
407  名不经传
408  默默耕耘
409  曾经默默

[410 rows x 1 columns]


In [14]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
SES_unique_similar_words_list = list(set(SES_similar_words_list))

In [15]:
# 将去重后的结果转换为 DataFrame
df_unique_SES_similar_words = pd.DataFrame(SES_unique_similar_words_list, columns=['Word'])
print(df_unique_SES_similar_words)

         Word
0       富足的生活
1        身价飙升
2        鲜花着锦
3         很气派
4         土豪们
..        ...
358       不体面
359      默默无名
360  无人不知无人不晓
361      人生顶峰
362      比别人强

[363 rows x 1 columns]


In [16]:
# 保存去重后的相似词表到excel文件
df_unique_SES_similar_words.to_excel(r'result\SES_unique_positive_similar_words.xlsx', index=False)

In [17]:
# 消极词汇
df_SES_negative = df_SES_words.loc[df_SES_words['Valence']=='Negative', 'Word']
df_SES_negative.head()

41       穷
42    家徒四壁
43    财大气粗
44      贫穷
45      贫困
Name: Word, dtype: object

In [18]:
# 格式转化为列表
SES_negative_word_list = df_SES_negative.tolist()
SES_negative_word_list

['穷',
 '家徒四壁',
 '财大气粗',
 '贫穷',
 '贫困',
 '一贫如洗',
 '贫苦',
 '底层',
 '穷困',
 '穷苦',
 '低层',
 '一穷二白',
 '高不可攀',
 '下等',
 '贫寒',
 '一无所有',
 '清贫',
 '潦倒',
 '低人一等',
 '低等',
 '寒酸',
 '奢糜',
 '穷酸',
 '窘困',
 '无名小卒',
 '挥金如土',
 '弱势',
 '奢华',
 '困苦',
 '身败名裂',
 '官僚',
 '贫乏',
 '卑贱',
 '低贱',
 '债台高筑',
 '奢靡',
 '清寒',
 '人微言轻',
 '困窘',
 '养尊处优',
 '拮据',
 '卑微',
 '寒碜',
 '落魄',
 '窘迫',
 '一文不名',
 '囊中羞涩',
 '草根',
 '奢侈',
 '低三下四',
 '穷乏',
 '低端',
 '摆阔',
 '清苦',
 '微贱',
 '不名一文',
 '轻贱',
 '低档',
 '微不足道',
 '倾家荡产',
 '卑下',
 '困难',
 '惟我独尊',
 '独裁',
 '无足轻重',
 '低声下气',
 '凄惨',
 '鼠辈',
 '艰苦',
 '挥霍',
 '游手好闲',
 '虚名']

In [19]:
# 初始化有效种子词列表和相似词列表
SES_negative_word_list_effective = []
SES_similar_negative_words_list = []

# 过滤词向量模型中存在的词并生成相似词
for word in SES_negative_word_list:
    if word in tencent_embedding:
        SES_negative_word_list_effective.append(word)
        # 找到与该词最相似的 10 个词
        SES_similar_negative_words = tencent_embedding.most_similar(word, topn=10)
        # 将相似词加入列表
        for similar_negative_word, similarity in SES_similar_negative_words:
            SES_similar_negative_words_list.append(similar_negative_word)
    else:
        print(f"Skipping {word}...")

Skipping 奢糜...


In [20]:
# 将生成的消极相似词转换为 DataFrame，查看是否生成成功
df_SES_similar_negative_words = pd.DataFrame(SES_similar_negative_words_list, columns=['Word'])
print(df_SES_similar_negative_words)

     Word
0      穷的
1      穷啊
2      穷了
3      很穷
4      是穷
..    ...
705    利禄
706  功名利禄
707    虚衔
708  空有虚名
709  身外之物

[710 rows x 1 columns]


In [22]:
# 将相似词列表转换为集合再转换回列表，以去掉重复出现的词
SES_unique_similar_negative_words_list = list(set(SES_similar_negative_words_list))

# 将去重后的结果转换为 DataFrame
df_SES_unique_similar_negative_words = pd.DataFrame(SES_unique_similar_negative_words_list, columns=['Word'])
print(df_SES_unique_similar_negative_words)

     Word
0      骄奢
1      底层
2      穷酸
3     太寒酸
4    名声败坏
..    ...
543    紧巴
544  巨额债务
545  摇尾乞怜
546  被瞧不起
547   穷光蛋

[548 rows x 1 columns]


In [23]:
# 保存去重后的相似词表到excel文件
df_SES_unique_similar_negative_words.to_excel(r'result\SES_unique_negative_similar_words.xlsx', index=False)